In [1]:
import numpy as np
import math;
import pickle
import pandas as pd
from collections import OrderedDict
import importlib
import time
import timeit

import torch
import torch.nn
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torchvision.models.resnet import *
from torch.autograd import Variable
from torchvision import transforms

import NNs
from NNs import *
importlib.reload(NNs)
import math
from NNs import ResNetDynamic, FeatureBoostedCNN

import glob
import cv2

from torchsummary import summary
from Preprocessing import *
from Preprocessing import ListsTrainDataset, ListsTestDataset

In [2]:
import sys
print(sys.path)

['/home/dimtsi/Dropbox/UvA/1st Semester/Applied Machine Learning/Project', '/home/dimtsi/anaconda3/envs/torch/lib/python37.zip', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7/lib-dynload', '', '/home/dimtsi/.local/lib/python3.7/site-packages', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7/site-packages', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7/site-packages/IPython/extensions', '/home/dimtsi/.ipython']


## LOAD DATA

In [3]:
original_train_images = pickle.load(open("pkl/extraclassified_padded64.pkl", "rb"))
# train_images = train_images[:1000]
original_labels = pickle.load(open("pkl/train_labels.pkl", "rb"))
kaggle_test_images = pickle.load(open("pkl/test_padded64.pkl", "rb"))
kaggle_test_filenames = pickle.load(open("pkl/test_filenames.pkl", "rb"))

## Load handcrafted features

In [4]:
original_haralick = pickle.load(open("features/extraclassified/train_haralick.pkl", "rb"))
original_moments = pickle.load(open("features/extraclassified/train_moments.pkl", "rb"))
original_sizes = pickle.load(open("features/extraclassified/train_sizes.pkl", "rb"))

kaggle_test_haralick = pickle.load(open("features/extraclassified/test_haralick.pkl", "rb"))
kaggle_test_moments = pickle.load(open("features/extraclassified/test_moments.pkl", "rb"))
kaggle_test_sizes = pickle.load(open("features/extraclassified/test_sizes.pkl", "rb"))

train_handcrafted_features = np.concatenate([original_haralick, original_moments,  original_sizes], axis =1)
kaggle_test_handcrafted_features = np.concatenate([kaggle_test_haralick, kaggle_test_moments,  kaggle_test_sizes], axis =1)

## Split to train test mine

In [61]:
test_set_mine_indexes = pickle.load(open("pkl/test_set_mine_indexes_extraclassified.pkl", "rb"))

train_images = [i for j, i in enumerate(original_train_images) if j not in test_set_mine_indexes]
train_labels = [i for j, i in enumerate(original_labels) if j not in test_set_mine_indexes]
train_handcrafted = [i for j, i in enumerate(train_handcrafted_features) if j not in test_set_mine_indexes]

#
test_mine_images = [i for j, i in enumerate(original_train_images) if j in test_set_mine_indexes]
test_mine_labels = [i for j, i in enumerate(original_labels) if j in test_set_mine_indexes]
test_mine_handcrafted = [i for j, i in enumerate(train_handcrafted_features) if j in test_set_mine_indexes]

In [56]:
X_train_cnn, y_train_cnn = train_images, train_labels
X_val_cnn, y_val_cnn = test_mine_images, test_mine_labels

## CNN

In [44]:
pretrained = resnet50(pretrained = True)
cnn = ResNetDynamic(pretrained.block, pretrained.layers, num_layers = 2, pretrained_nn = None)
#
cnn_dict = torch.load('models/extraclassified/trained_model90.pt', map_location={"cuda:1": "cuda:0", "cuda:2": "cuda:0"})['state_dict']
cnn.load_state_dict(cnn_dict)
cnn = cnn.eval().cuda()
del(pretrained)
feature_extractor_cnn = nn.Sequential(*list(cnn.children())[:-2]).eval().cuda()

In [46]:
mean_norm_test, std_norm_test = calc_means_stds(train_images)

def get_cnn_features(feature_extractor, model, x):
    features = ...
    mean_norm_test, std_norm_test = calc_means_stds(train_images)

    test_transforms = transforms. Compose([
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean_norm_test],
                    std =[std_norm_test])
    ])
    
    total_features = torch.Tensor().float().cpu()
    total_predicted = torch.Tensor().long()
    total_probabilities = torch.Tensor().float()  
    
    cnn_dataset = ListsTestDataset(x, transform = test_transforms)
    cnn_loader = torch.utils.data.DataLoader(cnn_dataset, batch_size = 32, shuffle = False)

    predictions = []
    for i, images in enumerate(cnn_loader):
        images = Variable(images, requires_grad=False).cuda()
        outputs = model(images).cuda()
        _, predicted = torch.max(outputs.data, 1)
        features = feature_extractor(images)
        
        total_features = torch.cat((total_features, features.detach().cpu()))
        total_predicted = torch.cat((total_predicted, predicted.cpu().long()))
        total_probabilities = torch.cat((total_probabilities,(torch.nn.Softmax()(outputs)).detach().cpu()))

    return (total_features.numpy(), total_predicted.numpy(), total_probabilities.numpy())

In [57]:
cnn_train_features, cnn_train_predictions, cnn_train_probabilities = get_cnn_features(feature_extractor_cnn, cnn, X_train_cnn)
cnn_val_features, cnn_val_predictions, cnn_val_probabilities = get_cnn_features(feature_extractor_cnn, cnn, X_val_cnn)
cnn_kaggle_features, cnn_kaggle_predictions, cnn_kaggle_probabilities = get_cnn_features(feature_extractor_cnn, cnn, kaggle_test_images)

/home/dimtsi/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


## Scale and Preprocess for Ensemble

In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_handcrafted_train = scaler.fit_transform(train_handcrafted)
scaled_handcrafted_val = scaler.fit_transform(test_mine_handcrafted)
scaled_handcrafted_kaggle = scaler.fit_transform(kaggle_test_handcrafted_features)
scaled_cnn_train_features = scaler.fit_transform(cnn_train_features)
scaled_cnn_val_features = scaler.fit_transform(cnn_val_features)
scaled_cnn_kaggle_features = scaler.fit_transform(cnn_kaggle_features)

### Setup Features DF

In [77]:
feature_names = []

for i in range(original_haralick.shape[1]):
    feature_names.append("haralick"+str(i))
for i in range(original_moments.shape[1]):
    feature_names.append("moments"+str(i))
for i in range(original_sizes.shape[1]):
    feature_names.append("sizes"+str(i))
for i in range(scaled_cnn_train_features.shape[1]):
    feature_names.append("deep"+str(i))   
len(handcrafted_names)

551

In [80]:
##concat handcrafted and deep features

NP_FEATURES_TRAIN = np.concatenate([scaled_handcrafted_train, scaled_cnn_train_features], axis = 1)
NP_FEATURES_VAL = np.concatenate([scaled_handcrafted_val, scaled_cnn_val_features], axis = 1)
NP_FEATURES_KAGGLE = np.concatenate([scaled_handcrafted_kaggle, scaled_cnn_kaggle_features], axis = 1)

y_train = train_labels
y_test = test_mine_labels

X_train = pd.DataFrame(NP_FEATURES_TRAIN, columns = feature_names)
y_test = pd.DataFrame(NP_FEATURES_VAL, columns = feature_names)

X_train.head()

,haralick0,haralick1,haralick2,haralick3,haralick4,haralick5,haralick6,haralick7,haralick8,haralick9,...,deep502,deep503,deep504,deep505,deep506,deep507,deep508,deep509,deep510,deep511
0,0.451856,-0.231419,0.502691,-0.481117,0.387366,0.462307,-0.465115,-0.391775,-0.424662,0.436325,...,-0.411653,-0.359564,3.070508,-0.613161,-0.332969,-0.277463,-0.635314,-0.062498,-0.742805,-0.556487
1,-0.133810,-0.411046,1.080848,-0.594465,-0.030692,0.276956,-0.552396,0.143724,0.056350,-0.139571,...,1.769049,-0.664688,0.797521,-0.735451,-0.462577,-0.495096,0.845502,-0.642879,1.320392,-0.693464
2,0.318293,-0.187910,0.556161,-0.305631,0.218710,0.127789,-0.288176,-0.184745,-0.285997,0.248271,...,-0.367620,-0.664688,0.758128,-0.502803,-0.491951,-0.628677,-0.513375,-0.609936,-0.130464,-0.666112
3,-0.843213,-0.099349,0.885158,0.678521,-0.476688,-0.556088,0.731822,0.416865,0.301075,-0.804225,...,-0.406037,-0.664688,3.178635,0.890270,-0.491951,-0.628677,-0.218238,-0.642879,-0.394223,-0.717080
4,0.062575,-0.269070,0.263624,-0.719381,0.014959,0.530318,-0.709168,-0.042254,-0.096993,0.006578,...,2.360212,-0.664688,0.361874,-0.703543,-0.330049,0.137321,0.805877,-0.607581,2.206954,-0.651078


In [ ]:
train_dataset.features

In [ ]:
train_and_validate_with_features(ensemble_nn, train_loader, val_loader, num_epochs=100)

In [ ]:
handcrafted_val.shape